#### Import all the necessary modules

In [1]:
import pandas as pd
from sklearn.utils import resample
import io
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.metrics import silhouette_score, adjusted_rand_score
import time
import psutil
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors

from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

#### Read data from ransomware dataset

In [2]:
from google.colab import files
uploaded = files.upload()

Saving feature_reduced_df.csv to feature_reduced_df.csv


In [3]:
df = pd.read_csv(io.BytesIO(uploaded['feature_reduced_df.csv']))
df

,Characteristics,MajorLinkerVersion,SizeOfCode,SizeOfInitializedData,AddressOfEntryPoint,BaseOfData,ImageBase,MajorOperatingSystemVersion,MinorImageVersion,SizeOfImage,...,ExportNb,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,VersionInformationSize,legitimate
0,-0.894729,0.541940,0.258079,0.812257,0.302712,0.524661,0.0,-0.122970,-0.724506,0.443813,...,-0.681546,1.182562,2.870069,-0.192823,1.704346,1.076381,-0.527257,0.520126,-0.085086,0
1,0.937485,0.191129,-0.370913,-0.909323,-0.088360,-1.662472,0.0,1.148133,1.304312,-1.090428,...,1.120038,-0.031681,-0.352520,0.338727,-0.242545,-0.491967,0.517460,-0.641502,0.364544,1
2,0.937485,0.191129,0.846523,0.786809,-0.283639,-1.662472,0.0,1.148133,1.304312,0.674571,...,1.349836,1.426796,0.895758,0.314872,0.638564,0.487561,0.148302,1.169524,0.364544,1
3,-0.868401,-0.962072,-0.476705,0.122190,-0.386241,-0.033794,0.0,-1.157404,-0.724506,-0.276871,...,-0.681546,-0.031681,-0.539677,-0.539158,0.009378,-1.048423,-1.119224,-0.999491,-0.499706,0
4,0.951505,0.191129,0.662464,-0.212981,-0.396592,0.843933,0.0,1.148133,1.304312,0.079556,...,1.470306,-0.826127,-0.766255,0.050954,-1.038441,-0.860062,0.420891,-0.905301,0.364544,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82641,-0.894729,0.541940,0.264859,0.816562,0.316738,0.538923,0.0,-0.122970,-0.724506,0.455236,...,-0.681546,0.121885,-0.200550,-0.192823,0.046946,0.099037,-0.527257,0.170026,0.135465,0
82642,0.951505,0.191129,-2.225670,-1.090464,-2.278104,-0.592405,0.0,1.148133,1.304312,-1.858657,...,0.481717,-1.381261,-0.294803,1.290986,-1.032684,-0.569546,1.400497,-0.919070,0.364544,1
82643,0.937485,0.191129,-0.064729,-0.618476,-0.757003,-1.662472,0.0,1.148133,1.304312,-0.728444,...,0.900496,-0.474482,2.168491,1.285405,1.395959,-0.643216,1.286472,-0.921071,0.364544,1
82644,-0.894729,0.541940,0.251229,0.824249,0.292214,0.524661,0.0,-0.122970,-0.724506,0.443813,...,-0.681546,-0.031681,-0.562173,0.071521,-0.916827,0.046254,-0.527257,0.042503,-0.085086,0


In [4]:
feature_reduced_df = df.sample(frac=1, random_state=42).reset_index(drop=True)
feature_reduced_df.head()

,Characteristics,MajorLinkerVersion,SizeOfCode,SizeOfInitializedData,AddressOfEntryPoint,BaseOfData,ImageBase,MajorOperatingSystemVersion,MinorImageVersion,SizeOfImage,...,ExportNb,ResourcesNb,ResourcesMeanEntropy,ResourcesMinEntropy,ResourcesMaxEntropy,ResourcesMeanSize,ResourcesMinSize,ResourcesMaxSize,VersionInformationSize,legitimate
0,0.952246,-0.558059,0.180947,-0.862871,-0.168401,0.463351,0.0,-0.122970,1.304312,-0.598506,...,1.349836,-1.381261,-0.447579,1.073465,-1.126915,-0.613263,1.356297,-0.952159,0.364544,1
1,0.951505,-0.174725,-0.963464,-1.329581,-0.705194,-0.300109,0.0,-0.122970,-0.724506,-1.732128,...,1.747654,-0.826127,0.379913,1.234060,-0.157741,-0.733025,0.945957,-0.897611,0.601950,1
2,0.951505,0.541940,-0.250224,-0.217020,-0.740525,0.156952,0.0,1.148133,1.304312,-0.673643,...,1.616478,0.248808,-0.916403,-0.352887,-1.031232,-1.045415,-1.119224,-0.919070,0.364544,1
3,0.951505,0.191129,0.922639,-0.123685,0.311533,1.071969,0.0,1.148133,2.167675,0.364947,...,1.256403,-0.223563,0.754897,1.287026,-0.000828,-1.134109,0.090820,-0.923080,0.364544,1
4,-1.980270,0.191129,-0.391751,-0.116827,-0.039788,-1.662472,0.0,1.148133,1.304312,-0.598506,...,-0.681546,0.724498,1.134074,0.362914,1.691817,0.519298,0.214727,0.860269,0.364544,1


In [5]:
X = feature_reduced_df.drop('legitimate', axis=1)
y = feature_reduced_df['legitimate']

#### K Nearest Neighbour with K = 5

In [ ]:
# K Nearest Neighbour
# Initialize the KNN model with 2 neighbors
knn = KNeighborsClassifier(n_neighbors=2)

# Data Split
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
end_processing = time.time()

# Training
start_training = time.time()
knn.fit(X_train, y_train)
end_training = time.time()

# Testing / Prediction
start_testing = time.time()
y_pred = knn.predict(X_test)
end_testing = time.time()

# Accuracy and additional metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print time metrics
print("Processing Time: {:.4f} seconds".format(end_processing - start_processing))
print("Training Time: {:.4f} seconds".format(end_training - start_training))
print("Testing Time: {:.4f} seconds".format(end_testing - start_testing))

# Print performance metrics
print("Model Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

# Print confusion matrix and classification report
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)

Processing Time: 0.0429 seconds
Training Time: 0.0156 seconds
Testing Time: 13.1914 seconds
Model Accuracy: 0.9908
Precision: 0.9925
Recall: 0.9890
F1 Score: 0.9908

Confusion Matrix:
[[12289    93]
 [  136 12276]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     12382
           1       0.99      0.99      0.99     12412

    accuracy                           0.99     24794
   macro avg       0.99      0.99      0.99     24794
weighted avg       0.99      0.99      0.99     24794



### K Nearest Neighbour - 5 fold cross validation - Bayesian Optimization

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score
import numpy as np

# Define the search space for Bayesian Optimization
param_space = {
    'n_neighbors': (1, 20),  # Search space for n_neighbors between 1 and 20
    'weights': ['uniform', 'distance'],  # Search space for 'weights' hyperparameter
    'metric': ['euclidean', 'manhattan', 'minkowski']  # Distance metrics
}

# Define the KNN model
knn_model = KNeighborsClassifier()

# Define scoring metrics
scoring = {
    'accuracy': 'accuracy',
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

# Use Bayesian Optimization with cross-validation (5 folds)
opt = BayesSearchCV(
    estimator=knn_model,
    search_spaces=param_space,
    n_iter=30,  # Number of iterations of optimization
    cv=5,  # 5-fold cross-validation
    scoring='accuracy',  # You can use 'accuracy', 'precision', 'recall', 'f1', or a combination
    random_state=42,
    n_jobs=-1
)

# Split the dataset (you can skip this if you already have X_train and y_train)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model using Bayesian Optimization
opt.fit(X_train, y_train)

# Print the best parameters found by Bayesian Optimization
print("Best parameters found by Bayesian Optimization:", opt.best_params_)

# Evaluate the model with the best parameters
y_pred = opt.predict(X_test)

# Calculate evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Optimized KNN Accuracy: {accuracy:.4f}")
print(f"Optimized KNN Precision: {precision:.4f}")
print(f"Optimized KNN Recall: {recall:.4f}")
print(f"Optimized KNN F1 Score: {f1:.4f}")


Best parameters found by Bayesian Optimization: OrderedDict([('metric', 'manhattan'), ('n_neighbors', 6), ('weights', 'distance')])
Optimized KNN Accuracy: 0.9916
Optimized KNN Precision: 0.9879
Optimized KNN Recall: 0.9954
Optimized KNN F1 Score: 0.9916


### Support Vector Machine

In [ ]:
# Support Vector Machine

svm_model = SVC(kernel='linear', random_state=42)  # You can choose other kernels like 'rbf', 'poly', etc.

# Step 1: Split the data into training and testing sets
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
end_processing = time.time()

# Step 2: Measure training time and fit the model
start_training = time.time()
svm_model.fit(X_train, y_train)
end_training = time.time()

# Step 3: Measure testing time and make predictions
start_testing = time.time()
y_pred = svm_model.predict(X_test)
end_testing = time.time()

# Step 4: Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Step 5: Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Display the results
print("Processing Time: {:.4f} seconds".format(end_processing - start_processing))
print("Training Time: {:.4f} seconds".format(end_training - start_training))
print("Testing Time: {:.4f} seconds".format(end_testing - start_testing))
print("Model Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

# Print confusion matrix and classification report
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)

Processing Time: 0.1101 seconds
Training Time: 40.2783 seconds
Testing Time: 3.1074 seconds
Model Accuracy: 0.9837
Precision: 0.9869
Recall: 0.9806
F1 Score: 0.9837

Confusion Matrix:
[[12220   162]
 [  241 12171]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     12382
           1       0.99      0.98      0.98     12412

    accuracy                           0.98     24794
   macro avg       0.98      0.98      0.98     24794
weighted avg       0.98      0.98      0.98     24794



### Support Vector Machine with 5 fold cross validation - Bayesian Optimization

In [ ]:
from sklearn.svm import SVC
from skopt import BayesSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

# Initialize the SVM model (No need to specify parameters yet, as we will optimize them)
svm_model = SVC(random_state=42)

# Define the search space for hyperparameters
search_space = {
    'C': (1e-6, 100.0, 'log-uniform'),   # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'], # Kernel type
    'gamma': (1e-6, 1.0, 'log-uniform')  # Kernel coefficient for 'rbf' and 'poly'
}

# Set up Bayesian optimization with 5-fold cross-validation
opt = BayesSearchCV(
    estimator=svm_model,
    search_spaces=search_space,
    n_iter=30,  # Number of parameter settings to try
    cv=5,       # 5-fold cross-validation
    scoring='accuracy',  # You can also use 'precision', 'recall', or 'f1' as the primary metric
    random_state=42,
    n_jobs=-1   # Use all available cores for parallelization
)

# Fit the model with Bayesian optimization
opt.fit(X, y)

# Print the best hyperparameters found
print("Best Hyperparameters:", opt.best_params_)

# Evaluate the optimized model on cross-validation metrics
accuracy_scores = cross_val_score(opt.best_estimator_, X, y, cv=5, scoring='accuracy')
precision_scores = cross_val_score(opt.best_estimator_, X, y, cv=5, scoring=make_scorer(precision_score))
recall_scores = cross_val_score(opt.best_estimator_, X, y, cv=5, scoring=make_scorer(recall_score))
f1_scores = cross_val_score(opt.best_estimator_, X, y, cv=5, scoring=make_scorer(f1_score))

# Print mean and standard deviation of each metric across 5 folds
print("5-Fold Cross Validation Results After Bayesian Optimization:")
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_scores.mean(), accuracy_scores.std()))
print("Precision: {:.4f} ± {:.4f}".format(precision_scores.mean(), precision_scores.std()))
print("Recall: {:.4f} ± {:.4f}".format(recall_scores.mean(), recall_scores.std()))
print("F1 Score: {:.4f} ± {:.4f}".format(f1_scores.mean(), f1_scores.std()))


### Naive Bayes Model

In [ ]:
# Naive Bayes model

# Data Split
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
end_processing = time.time()

# Initialize the Naive Bayes model
nb_model = GaussianNB()

# Training
start_training = time.time()
nb_model.fit(X_train, y_train)
end_training = time.time()

# Testing / Prediction
start_testing = time.time()
y_pred = nb_model.predict(X_test)
end_testing = time.time()

# Accuracy and additional metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print time metrics
print("Processing Time: {:.4f} seconds".format(end_processing - start_processing))
print("Training Time: {:.4f} seconds".format(end_training - start_training))
print("Testing Time: {:.4f} seconds".format(end_testing - start_testing))

# Print performance metrics
print("Model Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

# Print confusion matrix and classification report
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)

Processing Time: 0.0612 seconds
Training Time: 0.0496 seconds
Testing Time: 0.0125 seconds
Model Accuracy: 0.9771
Precision: 0.9674
Recall: 0.9874
F1 Score: 0.9773

Confusion Matrix:
[[11969   413]
 [  156 12256]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.97      0.98     12382
           1       0.97      0.99      0.98     12412

    accuracy                           0.98     24794
   macro avg       0.98      0.98      0.98     24794
weighted avg       0.98      0.98      0.98     24794



### Naive Bayes with 5 fold cross validation and Bayesian Optimization

In [ ]:
from sklearn.naive_bayes import GaussianNB
from skopt import BayesSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score

# Initialize the Naive Bayes model
nb_model = GaussianNB()

# Define the search space for hyperparameter tuning
search_space = {
    'var_smoothing': (1e-9, 1e-2, 'log-uniform')  # Tuning var_smoothing on a logarithmic scale
}

# Set up Bayesian Optimization with 5-fold cross-validation
opt = BayesSearchCV(
    estimator=nb_model,
    search_spaces=search_space,
    n_iter=30,  # Number of parameter settings to try
    cv=5,       # 5-fold cross-validation
    scoring='accuracy',  # You can also use 'precision', 'recall', or 'f1' as the primary metric
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Fit the model with Bayesian optimization
opt.fit(X, y)

# Print the best hyperparameters found
print("Best Hyperparameters:", opt.best_params_)

# Evaluate the optimized model on cross-validation metrics
accuracy_scores = cross_val_score(opt.best_estimator_, X, y, cv=5, scoring='accuracy')
precision_scores = cross_val_score(opt.best_estimator_, X, y, cv=5, scoring=make_scorer(precision_score))
recall_scores = cross_val_score(opt.best_estimator_, X, y, cv=5, scoring=make_scorer(recall_score))
f1_scores = cross_val_score(opt.best_estimator_, X, y, cv=5, scoring=make_scorer(f1_score))

# Print mean and standard deviation of each metric across 5 folds
print("5-Fold Cross Validation Results After Bayesian Optimization:")
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_scores.mean(), accuracy_scores.std()))
print("Precision: {:.4f} ± {:.4f}".format(precision_scores.mean(), precision_scores.std()))
print("Recall: {:.4f} ± {:.4f}".format(recall_scores.mean(), recall_scores.std()))
print("F1 Score: {:.4f} ± {:.4f}".format(f1_scores.mean(), f1_scores.std()))


5-Fold Cross Validation Results:
Accuracy: 0.9782 ± 0.0016
Precision: 0.9702 ± 0.0023
Recall: 0.9867 ± 0.0013
F1 Score: 0.9784 ± 0.0016


### Logistic Regression

In [ ]:
# Function for running logistic regression
def log_regression(df, test_ratio):
  X = df.drop('legitimate', axis=1)
  y = df['legitimate']

  # Measure processing time
  start_processing = time.time()
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_ratio, random_state=42)
  end_processing = time.time()

  # Initialize and train the logistic regression model
  logreg = LogisticRegression()

  # Measure training time
  start_training = time.time()
  logreg.fit(X_train, y_train)
  end_training = time.time()

  # Measure testing/prediction time
  start_testing = time.time()
  y_pred = logreg.predict(X_test)
  end_testing = time.time()

  # Accuracy, confusion matrix, classification report
  accuracy = accuracy_score(y_test, y_pred)
  conf_matrix = confusion_matrix(y_test, y_pred)
  class_report = classification_report(y_test, y_pred)

  # Additional metrics (precision, recall, f1 score)
  precision = precision_score(y_test, y_pred)
  recall = recall_score(y_test, y_pred)
  f1 = f1_score(y_test, y_pred)

  # Print all metrics
  print(f"Processing Time: {end_processing - start_processing:.4f} seconds")
  print(f"Training Time: {end_training - start_training:.4f} seconds")
  print(f"Testing Time: {end_testing - start_testing:.4f} seconds")
  print(f"Model Accuracy: {accuracy:.4f}")

  # Print additional metrics
  print(f"Precision: {precision:.4f}")
  print(f"Recall: {recall:.4f}")
  print(f"F1 Score: {f1:.4f}")

  # Print confusion matrix and classification report
  print("\nConfusion Matrix:")
  print(conf_matrix)

  print("\nClassification Report:")
  print(class_report)

In [ ]:
# Performing logistic regression on the feature reduced dataframe using 30% test split and 40% test split to see if there is a difference
log_regression(feature_reduced_df,0.3)
print("-------------------------------")
log_regression(reduced_df,0.3)
print("-------------------------------")
# power_transformed_df['legitimate'] = transformed_df['legitimate']
# log_regression(power_transformed_df,0.3)
# log_regression(power_transformed_df,0.4)

Processing Time: 0.0984 seconds
Training Time: 0.7430 seconds
Testing Time: 0.0135 seconds
Model Accuracy: 0.9832
Precision: 0.9861
Recall: 0.9803
F1 Score: 0.9832

Confusion Matrix:
[[12211   171]
 [  245 12167]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     12382
           1       0.99      0.98      0.98     12412

    accuracy                           0.98     24794
   macro avg       0.98      0.98      0.98     24794
weighted avg       0.98      0.98      0.98     24794

-------------------------------


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Processing Time: 0.0690 seconds
Training Time: 4.2468 seconds
Testing Time: 0.0199 seconds
Model Accuracy: 0.9124
Precision: 0.9849
Recall: 0.8377
F1 Score: 0.9054

Confusion Matrix:
[[12223   159]
 [ 2014 10398]]

Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.99      0.92     12382
           1       0.98      0.84      0.91     12412

    accuracy                           0.91     24794
   macro avg       0.92      0.91      0.91     24794
weighted avg       0.92      0.91      0.91     24794

-------------------------------


### Logistic Regression with 5 fold cross validation and Bayesian Optimization

In [ ]:
from sklearn.linear_model import LogisticRegression
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, precision_score, recall_score, f1_score
import time

# Feature-reduced data (assuming feature_reduced_df is already created)
X = feature_reduced_df.drop('legitimate', axis=1)
y = feature_reduced_df['legitimate']

# Measure processing time
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
end_processing = time.time()

# Define the search space for Bayesian Optimization
search_space = {
    'C': (1e-6, 100.0, 'log-uniform'),  # Regularization strength
    'penalty': ['l2'],  # Regularization type (can also try 'l1' or 'elasticnet' if solver supports it)
    'solver': ['liblinear', 'lbfgs']  # Solver type
}

# Initialize logistic regression model
logreg = LogisticRegression()

# Set up Bayesian Optimization with 5-fold cross-validation
opt = BayesSearchCV(
    estimator=logreg,
    search_spaces=search_space,
    n_iter=30,  # Number of parameter settings to sample
    cv=5,       # 5-fold cross-validation
    scoring='accuracy',  # You can change this to precision, recall, or F1 if you prefer
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Fit the Bayesian optimization model on the training set
start_cv = time.time()
opt.fit(X_train, y_train)
end_cv = time.time()

# Get the best hyperparameters
print("Best Hyperparameters:", opt.best_params_)

# Measure training time
start_training = time.time()
logreg_optimized = opt.best_estimator_  # Best logistic regression model from Bayesian optimization
logreg_optimized.fit(X_train, y_train)
end_training = time.time()

# Measure testing/prediction time
start_testing = time.time()
y_pred = logreg_optimized.predict(X_test)
end_testing = time.time()

# Accuracy, confusion matrix, classification report
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Additional metrics (precision, recall, f1 score)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print cross-validation and timing results
print(f"Cross-validation Time: {end_cv - start_cv:.4f} seconds")
print(f"Processing Time: {end_processing - start_processing:.4f} seconds")
print(f"Training Time: {end_training - start_training:.4f} seconds")
print(f"Testing Time: {end_testing - start_testing:.4f} seconds")

# Print metrics
print(f"Model Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Print confusion matrix and classification report
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)


Cross-validation scores (Accuracy for each fold): [0.98349322 0.98444387 0.98228176 0.98444252 0.98107174]
Mean CV Accuracy: 0.9831
Standard Deviation of CV Accuracy: 0.0013
CV Time: 2.2272 seconds
Processing Time: 0.0795 seconds
Training Time: 0.3056 seconds
Testing Time: 0.0063 seconds
Model Accuracy: 0.9832
Precision: 0.9861
Recall: 0.9803
F1 Score: 0.9832

Confusion Matrix:
[[12211   171]
 [  245 12167]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     12382
           1       0.99      0.98      0.98     12412

    accuracy                           0.98     24794
   macro avg       0.98      0.98      0.98     24794
weighted avg       0.98      0.98      0.98     24794



### Decision Tree

In [ ]:
# Decision Tree

decision_tree = DecisionTreeClassifier(random_state=42)

# Step 1: Split the data into training and testing sets
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
end_processing = time.time()

# Step 2: Measure training time and fit the model
start_training = time.time()
decision_tree.fit(X_train, y_train)
end_training = time.time()

# Step 3: Measure testing time and make predictions
start_testing = time.time()
y_pred = decision_tree.predict(X_test)
end_testing = time.time()

# Step 4: Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Step 5: Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Display the results
print("Processing Time: {:.4f} seconds".format(end_processing - start_processing))
print("Training Time: {:.4f} seconds".format(end_training - start_training))
print("Testing Time: {:.4f} seconds".format(end_testing - start_testing))
print("Model Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

# Print confusion matrix and classification report
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)

Processing Time: 0.0431 seconds
Training Time: 2.0102 seconds
Testing Time: 0.0060 seconds
Model Accuracy: 0.9904
Precision: 0.9877
Recall: 0.9932
F1 Score: 0.9905

Confusion Matrix:
[[8169  102]
 [  56 8203]]

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      8271
           1       0.99      0.99      0.99      8259

    accuracy                           0.99     16530
   macro avg       0.99      0.99      0.99     16530
weighted avg       0.99      0.99      0.99     16530



### Decision Tree with 5 fold cross validation and Bayesian Optimization

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from skopt import BayesSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
import time

# Feature-reduced data (assuming feature_reduced_df is already created)
X = feature_reduced_df.drop('legitimate', axis=1)
y = feature_reduced_df['legitimate']

# Measure processing time
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
end_processing = time.time()

# Define the search space for Bayesian Optimization
search_space = {
    'max_depth': (1, 50),  # Maximum depth of the tree
    'min_samples_split': (2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': (1, 20),   # Minimum number of samples required to be at a leaf node
    'criterion': ['gini', 'entropy']  # Splitting criteria
}

# Initialize the Decision Tree model
decision_tree = DecisionTreeClassifier(random_state=42)

# Set up Bayesian Optimization with 5-fold cross-validation
opt = BayesSearchCV(
    estimator=decision_tree,
    search_spaces=search_space,
    n_iter=30,  # Number of iterations for Bayesian Optimization
    cv=5,       # 5-fold cross-validation
    scoring='accuracy',  # You can change this to precision, recall, or F1 if needed
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Perform Bayesian Optimization and fit the model on the training set
start_cv = time.time()
opt.fit(X_train, y_train)
end_cv = time.time()

# Get the best hyperparameters
print("Best Hyperparameters:", opt.best_params_)

# Evaluate the optimized model on the test set
decision_tree_optimized = opt.best_estimator_
y_pred = decision_tree_optimized.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print results
print(f"Cross-validation Time: {end_cv - start_cv:.4f} seconds")
print(f"Processing Time: {end_processing - start_processing:.4f} seconds")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


5-Fold Cross Validation Results:
Accuracy: 0.9907 ± 0.0005
Precision: 0.9896 ± 0.0012
Recall: 0.9919 ± 0.0007
F1 Score: 0.9907 ± 0.0005


### Random Forest Classifier - eventually used for bagging

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import time

# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Split the data into training and testing sets (70% training, 30% testing)
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
end_processing = time.time()

# Train the Random Forest model
start_training = time.time()
rf_model.fit(X_train, y_train)
end_training = time.time()

# Make predictions on the test set
start_testing = time.time()
y_pred = rf_model.predict(X_test)
end_testing = time.time()

# Calculate performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Print time metrics
print("Processing Time: {:.4f} seconds".format(end_processing - start_processing))
print("Training Time: {:.4f} seconds".format(end_training - start_training))
print("Testing Time: {:.4f} seconds".format(end_testing - start_testing))

# Print performance metrics
print("Model Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

# Print confusion matrix and classification report
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)


Processing Time: 0.0381 seconds
Training Time: 18.1964 seconds
Testing Time: 0.1556 seconds
Model Accuracy: 0.9936
Precision: 0.9923
Recall: 0.9950
F1 Score: 0.9937

Confusion Matrix:
[[8207   64]
 [  41 8218]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      8271
           1       0.99      1.00      0.99      8259

    accuracy                           0.99     16530
   macro avg       0.99      0.99      0.99     16530
weighted avg       0.99      0.99      0.99     16530



### Random forest with 5 fold cross validation and bayesian optimization

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from skopt import BayesSearchCV
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time

# Feature-reduced data (assuming feature_reduced_df is already created)
X = feature_reduced_df.drop('legitimate', axis=1)
y = feature_reduced_df['legitimate']

# Measure processing time
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
end_processing = time.time()

# Define the hyperparameter search space for Bayesian Optimization
search_space = {
    'n_estimators': (50, 300),  # Number of trees in the forest
    'max_depth': (5, 50),  # Maximum depth of the tree
    'min_samples_split': (2, 20),  # Minimum number of samples required to split a node
    'min_samples_leaf': (1, 20),   # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2'],  # Number of features to consider for the best split
    'bootstrap': [True, False]  # Whether bootstrap samples are used when building trees
}

# Initialize the Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Set up Bayesian Optimization with 5-fold cross-validation
opt = BayesSearchCV(
    estimator=rf_model,
    search_spaces=search_space,
    n_iter=30,  # Number of iterations for Bayesian Optimization
    cv=5,       # 5-fold cross-validation
    scoring='accuracy',  # You can change this to precision, recall, or F1 if needed
    random_state=42,
    n_jobs=-1   # Use all available cores
)

# Perform Bayesian Optimization and fit the model on the training set
start_cv = time.time()
opt.fit(X_train, y_train)
end_cv = time.time()

# Get the best hyperparameters
print("Best Hyperparameters:", opt.best_params_)

# Evaluate the optimized model on the test set
rf_optimized = opt.best_estimator_
y_pred = rf_optimized.predict(X_test)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print results
print(f"Cross-validation Time: {end_cv - start_cv:.4f} seconds")
print(f"Processing Time: {end_processing - start_processing:.4f} seconds")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


5-Fold Cross Validation Results:
Accuracy: 0.9941 ± 0.0005
Precision: 0.9928 ± 0.0008
Recall: 0.9954 ± 0.0007
F1 Score: 0.9941 ± 0.0005


### Stochastic Gradient Model

In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import time

# Initialize the SGD Classifier
sgd_model = SGDClassifier(random_state=42)

# Step 1: Split the data into training and testing sets (70% training, 30% testing)
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
end_processing = time.time()

# Step 2: Measure training time and fit the model
start_training = time.time()
sgd_model.fit(X_train, y_train)
end_training = time.time()

# Step 3: Measure testing time and make predictions
start_testing = time.time()
y_pred = sgd_model.predict(X_test)
end_testing = time.time()

# Step 4: Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Step 5: Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Display the results
print("Processing Time: {:.4f} seconds".format(end_processing - start_processing))
print("Training Time: {:.4f} seconds".format(end_training - start_training))
print("Testing Time: {:.4f} seconds".format(end_testing - start_testing))
print("Model Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

# Print confusion matrix and classification report
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)


Processing Time: 0.0766 seconds
Training Time: 0.3146 seconds
Testing Time: 0.0119 seconds
Model Accuracy: 0.9843
Precision: 0.9850
Recall: 0.9837
F1 Score: 0.9843

Confusion Matrix:
[[8147  124]
 [ 135 8124]]

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98      8271
           1       0.98      0.98      0.98      8259

    accuracy                           0.98     16530
   macro avg       0.98      0.98      0.98     16530
weighted avg       0.98      0.98      0.98     16530



In [ ]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Initialize the SGD Classifier
sgd_model = SGDClassifier(random_state=42)

# Perform 5-fold cross-validation
# Accuracy score
accuracy_scores = cross_val_score(sgd_model, X, y, cv=5, scoring='accuracy')
# Precision score
precision_scores = cross_val_score(sgd_model, X, y, cv=5, scoring=make_scorer(precision_score))
# Recall score
recall_scores = cross_val_score(sgd_model, X, y, cv=5, scoring=make_scorer(recall_score))
# F1 score
f1_scores = cross_val_score(sgd_model, X, y, cv=5, scoring=make_scorer(f1_score))

# Print mean and standard deviation of each metric across 5 folds
print("5-Fold Cross Validation Results:")
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_scores.mean(), accuracy_scores.std()))
print("Precision: {:.4f} ± {:.4f}".format(precision_scores.mean(), precision_scores.std()))
print("Recall: {:.4f} ± {:.4f}".format(recall_scores.mean(), recall_scores.std()))
print("F1 Score: {:.4f} ± {:.4f}".format(f1_scores.mean(), f1_scores.std()))


5-Fold Cross Validation Results:
Accuracy: 0.9830 ± 0.0015
Precision: 0.9855 ± 0.0015
Recall: 0.9803 ± 0.0032
F1 Score: 0.9829 ± 0.0016


### Boosted Tree XG Boost - used for boosting with hyperparameter tuning using Bayesian optimization

In [ ]:
import xgboost as xgb
from skopt import BayesSearchCV
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import time

# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)

# Step 1: Split the data into training and testing sets
start_processing = time.time()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
end_processing = time.time()

# Step 2: Define the hyperparameter search space for Bayesian Optimization
search_space = {
    'n_estimators': (50, 300),  # Number of trees in the forest
    'max_depth': (3, 10),       # Maximum depth of each tree
    'learning_rate': (0.01, 0.3, 'log-uniform'),  # Learning rate for boosting
    'subsample': (0.6, 1.0),    # Subsample ratio of the training data
    'colsample_bytree': (0.6, 1.0),  # Subsample ratio of features for each tree
    'min_child_weight': (1, 10),     # Minimum sum of instance weight (hessian) needed in a child
    'gamma': (0, 0.5),           # Minimum loss reduction required to make a further partition
}

# Step 3: Set up Bayesian Optimization with cross-validation
opt = BayesSearchCV(
    estimator=xgb_model,
    search_spaces=search_space,
    n_iter=30,          # Number of iterations for Bayesian Optimization
    cv=5,               # 5-fold cross-validation
    scoring='accuracy', # Use accuracy as the primary scoring metric
    random_state=42,
    n_jobs=-1           # Use all available cores
)

# Step 4: Measure the cross-validation time and fit the model
start_cv = time.time()
opt.fit(X_train, y_train)
end_cv = time.time()

# Step 5: Get the best hyperparameters found by Bayesian Optimization
print("Best Hyperparameters:", opt.best_params_)

# Step 6: Measure testing time and make predictions with the optimized model
start_testing = time.time()
y_pred = opt.best_estimator_.predict(X_test)
end_testing = time.time()

# Step 7: Calculate accuracy and other metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Step 8: Confusion matrix and classification report
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Display the results
print("Cross-validation Time: {:.4f} seconds".format(end_cv - start_cv))
print("Processing Time: {:.4f} seconds".format(end_processing - start_processing))
print("Testing Time: {:.4f} seconds".format(end_testing - start_testing))
print("Model Accuracy: {:.4f}".format(accuracy))
print("Precision: {:.4f}".format(precision))
print("Recall: {:.4f}".format(recall))
print("F1 Score: {:.4f}".format(f1))

# Print confusion matrix and classification report
print("\nConfusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)

Processing Time: 0.0423 seconds
Training Time: 1.4153 seconds
Testing Time: 0.0501 seconds
Model Accuracy: 0.9937
Precision: 0.9920
Recall: 0.9954
F1 Score: 0.9937

Confusion Matrix:
[[8205   66]
 [  38 8221]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      8271
           1       0.99      1.00      0.99      8259

    accuracy                           0.99     16530
   macro avg       0.99      0.99      0.99     16530
weighted avg       0.99      0.99      0.99     16530



In [ ]:
import xgboost as xgb
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Initialize the XGBoost model
xgb_model = xgb.XGBClassifier(random_state=42)

# Perform 5-fold cross-validation
# Accuracy score
accuracy_scores = cross_val_score(xgb_model, X, y, cv=5, scoring='accuracy')
# Precision score
precision_scores = cross_val_score(xgb_model, X, y, cv=5, scoring=make_scorer(precision_score))
# Recall score
recall_scores = cross_val_score(xgb_model, X, y, cv=5, scoring=make_scorer(recall_score))
# F1 score
f1_scores = cross_val_score(xgb_model, X, y, cv=5, scoring=make_scorer(f1_score))

# Print mean and standard deviation of each metric across 5 folds
print("5-Fold Cross Validation Results:")
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_scores.mean(), accuracy_scores.std()))
print("Precision: {:.4f} ± {:.4f}".format(precision_scores.mean(), precision_scores.std()))
print("Recall: {:.4f} ± {:.4f}".format(recall_scores.mean(), recall_scores.std()))
print("F1 Score: {:.4f} ± {:.4f}".format(f1_scores.mean(), f1_scores.std()))


5-Fold Cross Validation Results:
Accuracy: 0.9943 ± 0.0003
Precision: 0.9933 ± 0.0002
Recall: 0.9953 ± 0.0006
F1 Score: 0.9943 ± 0.0003


# Model Performance Comparison

| **Model**                | **Processing Time (s)** | **Training Time (s)** | **Testing Time (s)** | **Accuracy** | **Precision** | **Recall** | **F1 Score** |
|--------------------------|-------------------------|-----------------------|----------------------|--------------|---------------|------------|--------------|
| **Random Forest**         | 0.0565                  | 16.8087               | 0.2872               | **0.9940**   | 0.9923        | 0.9958     | 0.9940       |
| **Boosted Tree (XGBoost)**| 0.0628                  | 2.9771                | 0.0821               | 0.9937       | 0.9920        | 0.9956     | 0.9938       |
| **Decision Tree**         | 0.0861                  | 3.3468                | 0.0127               | 0.9909       | 0.9893        | 0.9925     | 0.9909       |
| **K-Nearest Neighbour**   | 0.0618                  | 0.0206                | 13.3704              | 0.9908       | 0.9925        | 0.9890     | 0.9908       |
| **Support Vector Machine**| 0.0612                  | 69.0863               | 1.5229               | 0.9837       | 0.9869        | 0.9806     | 0.9837       |
| **Logistic Regression**   | 0.1077                  | 0.8609                | 0.0233               | 0.9832       | 0.9861        | 0.9803     | 0.9832       |
| **Stochastic Gradient**   | 0.0611                  | 0.2202                | 0.0052               | 0.9823       | 0.9847        | 0.9799     | 0.9823       |
| **Naive Bayes**           | 0.0503                  | 0.0464                | 0.0143               | 0.9771       | 0.9674        | 0.9874     | 0.9773       |



#### Stacking Classifier

In [7]:
from sklearn.ensemble import StackingClassifier, BaggingClassifier

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define base models
knn_model = KNeighborsClassifier(n_neighbors=5)
svm_model = SVC(kernel='linear', probability=True, random_state=42)  # For stacking, we need probability estimates
nb_model = GaussianNB()
logreg = LogisticRegression(random_state=42)
decision_tree = DecisionTreeClassifier(random_state=42)

# Define Stacking Classifier (with Logistic Regression as the meta-model)
stacking_model = StackingClassifier(
    estimators=[
        ('knn', knn_model),
        ('svm', svm_model),
        ('naive_bayes', nb_model),
        ('logreg', logreg),
        ('decision_tree', decision_tree)
    ],
    final_estimator=LogisticRegression(),  # Meta-model can be Logistic Regression or any other model
    cv=5  # Cross-validation folds for stacking
)

# Fit the Stacking model
stacking_model.fit(X_train, y_train)

# Make predictions with stacking
y_pred_stack = stacking_model.predict(X_test)

# Evaluate the Stacking model
accuracy_stack = accuracy_score(y_test, y_pred_stack)
precision_stack = precision_score(y_test, y_pred_stack)
recall_stack = recall_score(y_test, y_pred_stack)
f1_stack = f1_score(y_test, y_pred_stack)

print(f"Stacking Accuracy: {accuracy_stack}")
print(f"Stacking Precision: {precision_stack}")
print(f"Stacking Recall: {recall_stack}")
print(f"Stacking F1 Score: {f1_stack}")


Stacking Accuracy: 0.992377495462795
Stacking Precision: 0.9915833130031715
Stacking Recall: 0.9930368922550696
Stacking F1 Score: 0.9923095703125


#### Bagging with Random Forest

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# Initialize Random Forest model
rf_model = RandomForestClassifier(random_state=42)

# Perform 5-fold cross-validation
# Accuracy score
accuracy_scores = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy')
# Precision score
precision_scores = cross_val_score(rf_model, X, y, cv=5, scoring=make_scorer(precision_score))
# Recall score
recall_scores = cross_val_score(rf_model, X, y, cv=5, scoring=make_scorer(recall_score))
# F1 score
f1_scores = cross_val_score(rf_model, X, y, cv=5, scoring=make_scorer(f1_score))

# Print mean and standard deviation of each metric across 5 folds
print("5-Fold Cross Validation Results:")
print("Accuracy: {:.4f} ± {:.4f}".format(accuracy_scores.mean(), accuracy_scores.std()))
print("Precision: {:.4f} ± {:.4f}".format(precision_scores.mean(), precision_scores.std()))
print("Recall: {:.4f} ± {:.4f}".format(recall_scores.mean(), recall_scores.std()))
print("F1 Score: {:.4f} ± {:.4f}".format(f1_scores.mean(), f1_scores.std()))


5-Fold Cross Validation Results:
Accuracy: 0.9942 ± 0.0005
Precision: 0.9927 ± 0.0007
Recall: 0.9957 ± 0.0011
F1 Score: 0.9942 ± 0.0005


In [9]:
sum = 0
for i in accuracy_scores:
  sum = sum + i
bagging_technique_accuracy = sum/5


#### Boosting with XGBoost

In [11]:
!pip install xgboost

In [12]:
from xgboost import XGBClassifier
# Define XGBoost model
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

# Fit the XGBoost model
xgb_model.fit(X_train, y_train)

# Make predictions with Boosting
y_pred_boost = xgb_model.predict(X_test)

# Evaluate the Boosting model
accuracy_boost = accuracy_score(y_test, y_pred_boost)
precision_boost = precision_score(y_test, y_pred_boost)
recall_boost = recall_score(y_test, y_pred_boost)
f1_boost = f1_score(y_test, y_pred_boost)

print(f"Boosting (XGBoost) Accuracy: {accuracy_boost}")
print(f"Boosting Precision: {precision_boost}")
print(f"Boosting Recall: {recall_boost}")
print(f"Boosting F1 Score: {f1_boost}")


Boosting (XGBoost) Accuracy: 0.9932244404113733
Boosting Precision: 0.9925573450463641
Boosting Recall: 0.9937698509650623
Boosting F1 Score: 0.9931632279330973


#### Summary of results

In [14]:
print("\n--- Model Comparison in terms of accuracy ---")
print(f"Stacking Model Accuracy: {accuracy_stack}")
print(f"Bagging Model Accuracy: {bagging_technique_accuracy}")
print(f"Boosting Model Accuracy: {accuracy_boost}")


--- Model Comparison in terms of accuracy ---
Stacking Model Accuracy: 0.992377495462795
Bagging Model Accuracy: 0.994204198549346
Boosting Model Accuracy: 0.9932244404113733


Average Accuracy: 0.9931678859149106

In [ ]:
print("\n--- Model Comparison ---")
print(f"Stacking Model Accuracy: {accuracy_stack}")
print(f"Bagging Model Accuracy: {bagging_technique_accuracy}")
print(f"Boosting Model Accuracy: {accuracy_boost}")